In [2]:
pip install azure-cosmos

Note: you may need to restart the kernel to use updated packages.


In [3]:
COSMOS_ENDPOINT = ''
COSMOS_KEY = ''

DATABASE_NAME = 'Test'
CONTAINER_NAME = 'retail'

In [4]:
from azure.cosmos import CosmosClient, exceptions, PartitionKey

In [5]:
client = CosmosClient(COSMOS_ENDPOINT, COSMOS_KEY)

In [7]:
database = client.create_database_if_not_exists(id=DATABASE_NAME)

In [8]:
container = database.create_container_if_not_exists(
    id=CONTAINER_NAME,
    partition_key=PartitionKey(path="/id"),
    offer_throughput=400
)

In [12]:
documento = {
    "id": "1",
    "nombre": "Producto A",
    "descripcion": "Descripcion del producto A",
    "precio": 29.99,
    "categoria": "Electronica"

}
type(documento)

dict

In [13]:
container.create_item(body=documento)

CosmosResourceExistsError: (Conflict) Entity with the specified id already exists in the system., 
RequestStartTime: 2024-10-08T01:29:00.0890610Z, RequestEndTime: 2024-10-08T01:29:00.0943718Z,  Number of regions attempted:1
{"systemHistory":[{"dateUtc":"2024-10-08T01:28:08.4576296Z","cpu":0.244,"memory":422938828.000,"threadInfo":{"isThreadStarving":"False","threadWaitIntervalInMs":0.0339,"availableThreads":32765,"minThreads":64,"maxThreads":32767},"numberOfOpenTcpConnection":1132},{"dateUtc":"2024-10-08T01:28:18.4676418Z","cpu":0.193,"memory":422953800.000,"threadInfo":{"isThreadStarving":"False","threadWaitIntervalInMs":0.0301,"availableThreads":32765,"minThreads":64,"maxThreads":32767},"numberOfOpenTcpConnection":1125},{"dateUtc":"2024-10-08T01:28:28.4782561Z","cpu":0.798,"memory":422839540.000,"threadInfo":{"isThreadStarving":"False","threadWaitIntervalInMs":0.0378,"availableThreads":32764,"minThreads":64,"maxThreads":32767},"numberOfOpenTcpConnection":1125},{"dateUtc":"2024-10-0

In [11]:
query = 'SELECT * FROM c'

items = list(container.query_items(
    query=query,
    enable_cross_partition_query=True
))

print(items)

[{'id': '1', 'nombre': 'Producto A', 'descripcion': 'Descripcion del producto A', 'precio': 29.99, 'categoria': 'Electronica', '_rid': 'Nb4eAO2pNK0BAAAAAAAAAA==', '_self': 'dbs/Nb4eAA==/colls/Nb4eAO2pNK0=/docs/Nb4eAO2pNK0BAAAAAAAAAA==/', '_etag': '"030003d8-0000-0100-0000-670487f10000"', '_attachments': 'attachments/', '_ts': 1728350193}]
